### <b>Fastapi</b>
- Fastapi is your ASGI(Asynchronous server gateway interface) supported framework
- Asynchronous programming (async/await) to handle many concurrent requests efficiently.
- Automatic API documentation (SWAGGER http://host:port/docs)
- Pydantic library to automatically validate request and response datas

### <b>Uvicorn</b>
- Uvicorn is the server that acts as a bridge between your fastapi application and the browser
- Uvicorn reads the address(host) door(port) from browser request and delivers the request if person(app) is active(running: uvicorn main:app)
- Browser sends bytes → Uvicorn translates bytes to ASGI → ASGI call is made → FastAPI executes logic → Uvicorn converts ASGI to bytes -> sends bytes back

### <b>Python-dotenv</b>
- load_dotenv() injects values into the global OS environment (os.environ) by reading .env file
- dotenv_values() reads .env file and returns a local dictionary with values as key-value pair. When explicit configuration matters
- load_dotenv(find_dotenv()) - Walks up directories, Finds nearest .env

### <b>SQLAlchemy & Drivers (asyncpg, psycopg2 ...)</b>

#### SQLAlchemy

- SQLAlchemy is a package we used as the main center to connect to databases like postgres, mysql databases etc
- SQLAlchemy handles all Connection pooling, Transaction isolation, Concurrency safety, has ORM, used for relational databases.. and many more
- URL -> SQLAlchemy -> Engine -> Pool allocation -> Request arrival -> Engine -> Session -> Database -> Engine -> Close session
- SQLAlchemy does NOT open a database connection when you create the engine. It opens a connection only when the first query actually needs it.
    - Why lazy connections are important ??

        If SQLAlchemy were eager instead of lazy:
        
        - App startup would:
            - Open DB connections immediately
            - Fail if DB is temporarily down
        - You’d waste connections even if:
            - App never hits DB
            - Some routes don’t need DB
- Connections are only created when first query runs, when connections is required.
    - when there is session.execute(query), it says “I actually need a connection → open one now.”
- It will be empty pools if there is no requirements, Engine say “I know how to connect, but I haven’t connected yet.”

#### Driver

- Although we don't manually import drivers anywhere in code, we must need driver to connect to the appropriate databases
    - PostgreSQL (Async) - asyncpg - postgresql+asyncpg://
    - PostgreSQL (Sync) - psycopg2 - postgresql+psycopg2://
    - MySQL (Async) - aiomysql - mysql+aiomysql://
    - MySQL (Sync) - pymysql - mysql+pymysql://
- Its passed along with url, Engine decodes the driver

#### Engine
- Guard of the connection pool
- Manages the connetion pool
- Assigns tokens (session) to new requests.
- if 10 connections are there, each parallel reqeusts are passed to different connections hence different session. 11th parallel request has to wait to get a session until one of them releases their session.
- create engine globally instead of creating in each request inside our application
- Engine creation is expensive, behind the scenes:
    - DNS resolution - parsing the url
    - TCP, TLS handshake
    - Authentication - username password verification etc.
    - session initilization
    - kernel resources
- Engine creation might take around 10-100ms per connection, that X100 request = disaster. so create engine(guard) globally

#### Session

- It begins when a connection is established and ends upon disconnection, log out, or a timeout due to inactivity. 
- How session is created:
    - when you call a request:
        ```bash
        @app.get("/health/db")
        async def db_health_check(db: AsyncSession = Depends(get_db)):
            ...
        ```
    - It calls:
        ``` bash
        async def get_db():
            async with AsyncSessionLocal() as session:
                    yield session
        ```
    - Under the hood (simplified):
        ``` bash
        gen = get_db()
        session = await gen.__anext__()   # before yield
        #route executes here
        await gen.aclose()                # after response
        ```
- a work manager that
    - borrows db connection from a pool
    - tracks python object
    - decide when sql should be sent

#### Work flow:

- App starts
- Url read by engine in SQLAlchemy
- <b>Engine</b> created (guard hired)
- Pool initialized (tunnels mapped, not opened yet)
- Request arrives
- <b>Session</b> created (pass issued)
- Connection borrowed (tunnel opened or reused) (managed by Engine(guard))
- Query executed
- Request finishes
- Session closes
- Connection returned to pool
- Engine notes that this connection is free and can be used by others
- App shuts down
- Engine disposed
- All tunnels closed (Guard goes home)

#### Mental models:

- Engine = connection manager
- Pool = performance optimizer
- Session = request conversation
- Async = concurrency without blocking
- Multiple users ≠ multiple engines
- DB guarantees data safety, not SQLAlchemy

### <b>Alembic and sqlalchemy.orm</b>

#### ORM
- ```ORM``` - object relational mapper, its a tool, it maps between two worlds(database and class)
- used to map the object in python to tables in database
- we connect rows in database to objects in python
- reads rows in database table as corresponding orm model defined in python
- Define the ORM model in ```model.py``` files

    ##### DeclaraiveBase

    - we declare a class defined as orm model when we pass ```DeclarativeBase``` in that class argument
    - once DeclarativeBase is defined, it will store the metadata of the classes that inherits from DeclarativeBase
    - it will store table name, columns, column features etc in ```DecalarativeBase.metadata```
    - we use ```DeclarativeBase.metadata.create_all()``` method to programmatically create tables in database.
    - it is not migration-aware and does not track schema changes

    ##### Mapped and mapped_column()
    - ```Mapped``` and ```mapped_column()``` is a new feature to identify the type of attribute for python.
    - old approach we dont know the type of attribute till runtime. but typing method will initilize the type earlier than runtime
    - old style:
        - ```bash 
            id = Column(Integer, primary_key=True) 
            ```
        - From Python’s perspective: 
            ```bash 
            type(User.id) == # Column
            ```
        - But at runtime: 
            ```bash 
            user = User()
            type(user.id)  # int
            ```
        - Without Mapped:
            - IDE cannot help you
            - Type errors show up only at runtime
    - New style:
        - ```bash 
            id: Mapped[int] = mapped_column(primary_key=True) 
            ```
        - Mapped[int] tells Python “This attribute behaves like int when accessed”
        - mapped_column() tells SQLAlchemy primary key, nullable, index, constraints. 
        - Column type will be infered from Mapped[int]
        - with Mapped:
            - Type errors caught early
            - Large-team friendly

#### Alembic
- Use Alembic for migrating database schema and tracking schema versions
- alembic will store the changes in its file like git does.
- you can think alembic as git for databases.
- always keep database updations and creation seperate from app struture
- Schema creation and updates are kept separate so that application runtime remains safe and free from schema mutation logic
    
    What happens if we keep schema mutation logic in app, eg. on_startup?
    - App v1 runs → creates tables
    - You deploy v2 (new column added)
    - App starts again → create_all() does NOTHING
    - DB schema ≠ code schema
    - App crashes later

- manual intervenion of changing the names of tables in python or ORM model might cause issue while alembic migration and it might give wrong migration script
- always cross check migration script before migrating to actualy db
- in migration sript:
    - upgrade() → Applies the migration (creates tables, columns, indexes, etc.) when you run alembic upgrade head or alembic upgrade +1.
    - downgrade() → Reverses the migration. It's used when you run alembic downgrade -1 or alembic downgrade <previous_revision>.
    - for index operations, alembic follow a naming convention: ix_<schema>_<tablename>_<columnnames>, The name is long but valid


### <b>Repository Layer</b>

- Repository is the thin layer of codes that connects between our fastapi and the database.
- It will convert you python requirements to SQL and database rows to ORM objects
- Input to the repository will have Asyncsession and output will be Orm model objects
- Repository must never assume how session was created.
- Repositories SHOULD:
    - Accept AsyncSession
    - Return ORM objects
- Repositories SHOULD NOT:
    - Raise HTTPException
    - Use Depends
    - Check permissions
    - Validate input
    - Hash passwords
- Workflow: Router > Service > Repository > AsyncSession > Engine > PostgreSQL
- Even if someone changes the fastapi to some other frameworks, repository layers should work
- Session powers are seen in repository layer:
    - ```session.add()```
        - (INSERT)
        - here there is no insert done nor db called
        - session will say “I am now responsible for this object”, it will track this object, what table it belongs to, what columns changed
        - it will add to pending state
    - ```session.add_all()```
        - adds multiple objects at once, add_all(User1,User2)
    - ```session.delete()```
        - delete the object
    - ```session.commit()```
        - (INSERT / UPDATE / DELETE)
        - session flush + end transcation
        - converts python objects to sql
        - executes queries
        - save the DB work
        - connection is released after work
    - ```session.flush()```
        - sends sql (INSERT / UPDATE / DELETE)
        - doesn't commit
        - eg. if you need id which autogenerates before commit
        - ```bash
            user = User(email="a@test.com")
            db.add(user)
            await db.flush()

            print(user.id)  # ✅ AVAILABLE
            ```
    - ```session.refresh()```
        - updates in-memory orm object
        - select row again
        - it will fetch the defaults and autoincrements autogeneration to python object so that we can use in our code
        - the flow is from DB to python, flush is from python to DB.
    - ```session.rollback()```
        - one of the most important player in application sessions.
        - used when there is error
        - used when exceptions raised
    - ```session.execute()```
        - executes the sql query
        - execute() is generic SQL execution, not just SELECT
        - It can run:
            - SELECT
            - UPDATE
            - DELETE
            - raw SQL
            - bulk operations
    - ```session.get(user,1)```
        - used for primary key searches
        - in the above example, its equivalent to - SELECT * FROM users WHERE id = 1

- ```stmt``` inside execute():
    - eg. stmt = select(User).where(User.email == email)
    - its the query blueprint, converted to sql queries while commits
    - ```db.execute(stmt)```
        - session borrows connection
        - stmt compiled to SQL
        - SQL sent to DB
        - rows returned

- result after execute is not orm object, You must extract data explicitly.
    - ```result.scalar_one_or_none()```, only returns one object, zero none, else error
    - ```result.scalar_one()```, if exactly one, else error.
    - ```result.scalar()```, first row, first column
    - ```result.scalars().all()```, every row as List[Objects]
    - ```result.first()```, gets the first object even if there is list of objects(Avoid, return is not ORM)
    - ```result.all()```, list[Row] (Avoid, return is not ORM)
    

### <b>Service Layer</b>

- We use service layer to add the business logic
- It's different from router and repositories. Routers should be thin.
- Service layer should not:
    - Not an HTTP handler   (routers do that)
    - Not a DB query writer (repositories do that)
    - Not a request validator (schemas do that)
- Flow: router -> service -> repository -> DB
- If FastAPI disappeared tomorrow, this code would still make sense.
- Services should not know they’re being called from an API.

### <b>Router Layer</b>

#### Router
- Router reads the HTTP request and matches the correct application funciton and works between HTTP and domain logic.
- Decides which function runs for a request
- Routers Do:
    - Define endpoints (path + method)
    - Parse request body
    - Inject dependencies (Depends)
    - Call services
    - Convert domain errors → HTTP responses
- My FastAPI routers are thin HTTP handlers. They inject dependencies, call services, and translate domain errors into HTTP responses. All business logic and persistence are isolated below the HTTP layer.

#### Pydantic
- Ensure data correctness at runtime
- Make sure request and response are as per the business logic.
- Any data passing through is inspected, verified, and either corrected or rejected before it’s allowed in.
- When you inherit from ```BaseModel```, Pydantic generates runtime logic for your class.
- BaseModel is not any model like HTTP model or database model or request/response model. It is pure data logic.

### <b>Security Layer</b>

- we have two security systems:
   - JWT authentication (code)
   - session (db)
- While logging in we generate jwt token and save session data in db.
- incoming request will have this token along with it, every request extract the token by decoding it.
- after decoding, we will be getting session id and user id and token expiration from it.
- those session id is validated for session expriy and allows only user that is within the session exipry.

```bash
Incoming request
   ↓
Extract token
   ↓
jwt.decode()
   ├── invalid signature → reject
   ├── expired token → reject
   └── valid token
         ↓
      Extract session_id
         ↓
      Load session from DB
         ├── inactive → reject
         ├── expired → reject
         └── active
               ↓
         Allow request
```

- What the difference between authentication and authorization ??
   - authentication - asks who you are ?, eg. your ID to enter the office.
   - authorization - are you still valid ? eg. security checks system to verify if your ID is still valid to enter the office.

#### JWT

- JWT (JSON Web Token) is a signed token that proves who a user is. Helps with authentication.
- jwt token contains three section : header, payload, signature.
- Without JWT:
   - User logs in
   - Server must remember user for every request (session-heavy)
- With JWT:
   - User logs in once
   - Server gives a token
   - Client sends that token with every request
   - Server verifies the token and trusts the request

- A JWT looks like this: xxxxx.yyyyy.zzzzz
- Header contains:
   ```bash
   {
      "alg": "HS256",
      "typ": "JWT"
   }
   ```
   It says: Which algorithm is used, That this is a JWT
- Payload contains:
   ```bash
   {
      "sub": "123",
      "exp": 1700000000
   }
   ```
   It contains: user infos, expiry, session id, roles etc.
- Signature:
   - This is created using: header + payload + secret_key

- Why Exp in payload is critical ?
   - Token automatically expires
   - JWT library rejects expired tokens
   - Limits damage if token is stolen

- JWT does NOT:
   - Store login state
   - Automatically log users out
   - Revoke itself
- That’s why in real systems, JWT is combined with:
   - Database sessions
   - Refresh tokens
- We use JWT for simplicity and reduce heavy burden of hitting DB every time for a request in sessions.

#### <i><b>If we reduce the heaviness of Session in each request, why enterprise apps use both jwt and session ?</b></i>
- Without JWT
   - You would need:
      - DB lookup just to authenticate
      - DB lookup just to identify user
      - DB lookup just to validate session
- With JWT
   - You:
      - Skip DB for identity verification
      - Skip DB for expiry check
      - Only hit DB for authorization control
- And that DB query is:
   - Indexed
   - Simple
   - Cheap
- This is a huge difference under load.

- What JWT does BEFORE DB:
   - JWT allows you to reject most bad requests without touching DB:
      - Invalid signature ❌
      - Expired token ❌
      - Malformed token ❌
   - All rejected cryptographically, cheaply.
   - Only valid tokens reach the DB.
- What Session does AFTER JWT:
   - Session check answers: “Even though this token is valid, do we still allow it?”
   - This DB hit is:
      - One indexed lookup
      - Very cheap
      - Much lighter than full auth logic

#### Sessions

- session asks: Even if someone proves who they are, should we still trust them right now?
- part of authentication with more control over user
- session gives you:
   - Logout
   - Revocation
   - Multi-device tracking
   - Forced expiry
   - Admin control
- JWT answers: Who are you?
- Session answers: Are you still allowed?